In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import utils
import datasets
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, Activation , Flatten, Dense
from keras.layers import LSTM, GRU
from keras.layers import Deconv3D,UpSampling3D, Activation

In [ ]:
# some magic commands
# %load_ext autoreload
%matplotlib inline

In [ ]:
# load data and display random example
X=datasets.load_dataset("ShapeNet")
input_shape=X.shape[1:]
plt.imshow(random.choice(X))

In [ ]:
# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]

In [ ]:
# encoder network
print(X.shape)
input_tensor=Input(shape=input_shape);
cur_tensor=Conv2D(filters=conv_filter_count[0],kernel_size= (7,7))(input_tensor)
for i in range(1,6):
    cur_tensor=Conv2D(filters=conv_filter_count[i],kernel_size= (3,3))(cur_tensor)
    cur_tensor=MaxPooling2D()(cur_tensor)
    cur_tensor=Activation('relu')(cur_tensor)
    print(cur_tensor.shape)

In [6]:
# construct feature vector
cur_tensor=Flatten()(cur_tensor)
print(cur_tensor.shape)
cur_tensor=Dense(1024)(cur_tensor)
print(cur_tensor.shape)

(?, ?)
(?, 1024)


In [7]:
# rnn module
# %autoreload
gru_grid= utils.grid3D(GRU(1024))
grid_slice=utils.get_neighbors(gru_grid,[4,4,4])
gru_layer=GRU(1024)

In [ ]:
# decoder network
for i in range(6):
    cur_tensor=Deconv3D(filters=deconv_filter_count[i],kernel_size=(3,3,3))(cur_tensor)
    cur_tensor=UpSampling3D()(cur_tensor)
    cur_tensor=Activation('relu')(cur_tensor)
    print(cur_tensor.shape)
    
print(net.output.shape())

In [8]:
net = Model(input=input_tensor,output=cur_tensor)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [9]:
net.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
os.system("rm -rf ./logs/*")
tensor_board=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
net.fit(X,np.random.rand(len(X),1024),epochs=1,callbacks=[tensor_board])

Epoch 1/1
5/5 [==============================] - 82s 16s/step - loss: 4768.5200 - acc: 0.0000e+00
